In [10]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))
import scipy.io as sio
from scipy.misc import imread
import tensorflow as tf
import numpy as np
import pandas as pd
import scipy.io
import tqdm 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from xgboost import XGBClassifier
import random
from numpy.random import choice
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, roc_curve
import sklearn.pipeline as pipeline
import sklearn.preprocessing as preprocessing
from sklearn.model_selection import train_test_split

In [11]:
data_folder = 'K:\\sdp_dataset\\GPDSSyntheticSignatures4k\\bmgnet_features'

In [12]:
USER_KERNEL='rbf'
TRAIN_ITERATIONS = 1

In [4]:
data_f = pd.read_csv(os.path.join(data_folder,'data_features.csv'))
visual_f = pd.read_csv(os.path.join(data_folder,'visual_features.csv'))

# MODEL SELECTION & TRAINING

In [5]:
dev_exp_ratio = 0.30
sorted_id_list = np.sort(data_f['user_id'].unique())
dev_exp_splitter=int(len(sorted_id_list)*dev_exp_ratio)
dev_val_user_ids = sorted_id_list[:dev_exp_splitter]
exp_user_ids = sorted_id_list[dev_exp_splitter:]

In [6]:
dev_val_user_ids.shape

(1200,)

In [7]:
exp_user_ids.shape

(2800,)

In [113]:
fakes_preds = []
gens_preds = []

for fold in np.arange(0,TRAIN_ITERATIONS):
    # assert len(dev_val_user_ids)==581
    np.random.shuffle(dev_val_user_ids)
    dev_user_ids = dev_val_user_ids[0:1000]
    validation_user_ids = dev_val_user_ids[1000:len(dev_val_user_ids)]
    train_idx, test_idx = train_test_split(np.arange(1,25), train_size=0.5, test_size=0.5)

    dev_df = data_f.loc[data_f['user_id'].isin(dev_user_ids)]
    dev_vf = visual_f.loc[dev_df.index]
    val_df = data_f.loc[data_f['user_id'].isin(validation_user_ids)]
    val_vf = visual_f.loc[val_df.index]

    dev_df_gen = dev_df.loc[dev_df['fakeness']==0]
    dev_df_fake = dev_df.loc[dev_df['fakeness']==1]
    dev_df_gen_12 = dev_df_gen.loc[dev_df_gen['sig_id'].isin(train_idx)]
    dev_df_valid_12 = dev_df_gen.loc[dev_df_gen['sig_id'].isin(test_idx)]
    
    train_idx, test_idx = train_test_split(np.arange(1,25), train_size=0.5)
    val_df_gen = val_df.loc[val_df['fakeness']==0]
    val_df_fake = val_df.loc[val_df['fakeness']==1]
    val_df_gen_12 = val_df_gen.loc[val_df_gen['sig_id'].isin(train_idx)]
    val_df_valid_gen_12 = val_df_gen.loc[val_df_gen['sig_id'].isin(test_idx)]

    for user_id in tqdm.tqdm(validation_user_ids, ascii=True):
        clf = SVC(C=1,gamma='scale',class_weight='balanced', probability=False, kernel=USER_KERNEL)
        y_train = (pd.concat([val_df_gen_12.loc[val_df_gen_12['user_id']==user_id],dev_df_gen.loc[dev_df_gen['user_id']!=user_id]]))['user_id']==user_id
        X_train = visual_f.loc[y_train.index]  
        clf.fit(X_train, y_train)
        y_valid_fakes = val_df_fake.loc[(val_df_fake['user_id']==user_id)]
        X_valid_f = visual_f.loc[y_valid_fakes.index]
        fakes_preds.append(clf.decision_function(X_valid_f))
        y_valid_gens = val_df_valid_gen_12.loc[val_df_valid_gen_12['user_id']==user_id]
        X_valid_g = visual_f.loc[y_valid_gens.index]
        gens_preds.append(clf.decision_function(X_valid_g))

c:\users\mert\anaconda3\envs\ml_gpu\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


  0%|                                                                                          | 0/200 [00:00<?, ?it/s]

  0%|4                                                                                 | 1/200 [00:04<14:30,  4.37s/it]

  1%|8                                                                                 | 2/200 [00:11<16:43,  5.07s/it]

  2%|#2                                                                                | 3/200 [00:16<17:08,  5.22s/it]

  2%|#6                                                                                | 4/200 [00:21<16:37,  5.09s/it]

  2%|##                                                                                | 5/200 [00:26<16:12,  4.99s/it]

  3%|##4                                                

 32%|##########################3                                                      | 65/200 [05:45<11:20,  5.04s/it]

 33%|##########################7                                                      | 66/200 [05:50<10:54,  4.89s/it]

 34%|###########################1                                                     | 67/200 [05:54<10:40,  4.82s/it]

 34%|###########################5                                                     | 68/200 [06:00<10:59,  4.99s/it]

 34%|###########################9                                                     | 69/200 [06:05<11:01,  5.05s/it]

 35%|############################3                                                    | 70/200 [06:10<11:02,  5.10s/it]

 36%|############################7                                                    | 71/200 [06:14<10:15,  4.77s/it]

 36%|#############################1                                                   | 72/200 [06:21<11:16,  5.28s/it]

 36%|###########################

 66%|####################################################8                           | 132/200 [11:35<06:31,  5.75s/it]

 66%|#####################################################2                          | 133/200 [11:41<06:39,  5.96s/it]

 67%|#####################################################6                          | 134/200 [11:46<06:04,  5.52s/it]

 68%|######################################################                          | 135/200 [11:51<05:48,  5.36s/it]

 68%|######################################################4                         | 136/200 [11:55<05:30,  5.16s/it]

 68%|######################################################8                         | 137/200 [12:00<05:16,  5.02s/it]

 69%|#######################################################2                        | 138/200 [12:06<05:39,  5.47s/it]

 70%|#######################################################6                        | 139/200 [12:13<05:53,  5.79s/it]

 70%|###########################

100%|###############################################################################6| 199/200 [17:30<00:05,  5.30s/it]

100%|################################################################################| 200/200 [17:36<00:00,  5.33s/it]

# GLOBAL THRESHOLD SELECTION

In [114]:
flat_fakes_preds = np.expand_dims(np.array([item for sublist in fakes_preds for item in sublist]),axis=1)
flat_gens_preds = np.expand_dims(np.array([item for sublist in gens_preds for item in sublist]),axis=1)
all_preds = np.vstack((flat_fakes_preds,flat_gens_preds))
all_labels = np.vstack((np.zeros((flat_fakes_preds.shape[0],1)),np.ones((flat_gens_preds.shape[0],1))))

fpr,tpr,threshold = roc_curve(all_labels,all_preds)
fnr = 1 - tpr
EER = fpr[np.nanargmin(np.absolute((fnr - fpr)))]
eer_th = threshold[np.nanargmin(np.absolute((fnr - fpr)))]
print('EER_glob : ', EER*100,'\nEER_Threshold_glob : ', eer_th)
glob_th = eer_th

EER_glob :  6.866666666666667 
EER_Threshold_glob :  0.052685639549978625


In [115]:
assert len(fakes_preds)==len(gens_preds)
EER_accum=0
for idx,val in enumerate(fakes_preds):
    user_fakes_preds = np.expand_dims(np.array(fakes_preds[idx]),axis=1)
    user_gens_preds = np.expand_dims(np.array(gens_preds[idx]),axis=1)
    all_user_preds = np.vstack((user_fakes_preds,user_gens_preds))
    all_user_labels = np.vstack((np.zeros((user_fakes_preds.shape[0],1)),np.ones((user_gens_preds.shape[0],1)))) 
    fpr,tpr,threshold = roc_curve(all_user_labels,all_user_preds)
    fnr = 1 - tpr
    EER = fpr[np.nanargmin(np.absolute((fnr - fpr)))]
    EER_accum += EER
    
print('EER_user : ', (EER_accum*100)/len(fakes_preds)) 

EER_user :  3.3666666666666645


In [116]:
print(glob_th)

0.052685639549978625


In [141]:
gens_preds

[array([0.75917679, 0.80417627, 0.47700485, 0.77539793, 0.81372301,
        0.38081687, 0.93040636, 0.87893064, 0.43077642, 0.65935252,
        0.6191321 , 0.86089001]),
 array([ 0.89220408,  0.01214401,  0.30079024,  0.60514133, -0.17866352,
         1.12326073,  0.61549855,  0.60054791,  0.07937873,  0.21190815,
         0.48790799,  0.93583668]),
 array([ 0.88866347, -0.00865281,  0.95197012,  0.73823633,  0.81686603,
         0.72370537,  0.94868429,  0.64135805,  0.91189309,  1.01957412,
        -0.00301373,  0.80983327]),
 array([0.80531664, 0.12406867, 0.70396322, 0.52644376, 0.47614151,
        0.57308068, 1.09201121, 0.67386534, 0.64816097, 0.0954567 ,
        0.96421774, 0.88634586]),
 array([0.77659992, 0.62373912, 0.51278129, 0.5181336 , 0.59696371,
        0.62990328, 0.69839528, 0.57281992, 0.5062787 , 0.97428985,
        0.8438921 , 1.236647  ]),
 array([0.87620133, 0.76031703, 0.83421543, 0.6731954 , 0.28513457,
        1.36983034, 1.23337922, 0.51985332, 1.04830955, 0.

In [127]:
validation_user_ids

array([ 807,  789,  633,  700,  166,  862,   55,   20,  698,  326,  455,
        323,  421, 1134,  187,  419,  397, 1183,  103,  497,  389,  380,
        390,  206,  521,  882,  801,   82,  301, 1100,  117,  983,  198,
        162, 1039,   39,  131,  506,  235,  243, 1095, 1138,  663,  338,
        996, 1108,  644,  121,  153,  969,  705,  250,  559,   37,  470,
        168,  478,   33,  212,  372,  784,  647, 1106,  672, 1070,  209,
        482,  736,  510,  591, 1072,  829,  291, 1014,  503,   79,  538,
        652,  330,  796,  477, 1196,  512, 1125,  740,  499, 1005,  526,
        847,  623,  848,  625,  908,  604,  960, 1167,   96,  906,  875,
        627,  890,  642, 1169,  465, 1198,  702,  461,  328,  810,  485,
        363,  691,  562,  525,  619,  886,  163,   97,  452,  233,  654,
        941,    5,  409,  645,  656,  713,   16,  825,  724,  139,  275,
        312,    6, 1029,  108,  204,  634,  520,  205,   19, 1034,  513,
        354,  701,  249,   36,  660, 1042,  975,  9

In [139]:
user_id = 789
val_df_valid_gen_12.loc[val_df_valid_gen_12['user_id']==user_id]

,path,user_id,sig_id,fakeness
204607,c-789-02.jpg,789,2,0
204608,c-789-03.jpg,789,3,0
204609,c-789-04.jpg,789,4,0
204610,c-789-05.jpg,789,5,0
204611,c-789-06.jpg,789,6,0
204614,c-789-09.jpg,789,9,0
204617,c-789-12.jpg,789,12,0
204619,c-789-14.jpg,789,14,0
204620,c-789-15.jpg,789,15,0
204621,c-789-16.jpg,789,16,0


In [133]:
val_df.loc[[205578,205584,205585,205587,205588,205590,205594,205595,205596,205597,205598,205601]]

,path,user_id,sig_id,fakeness
205578,c-807-01.jpg,807,1,0
205584,c-807-07.jpg,807,7,0
205585,c-807-08.jpg,807,8,0
205587,c-807-10.jpg,807,10,0
205588,c-807-11.jpg,807,11,0
205590,c-807-13.jpg,807,13,0
205594,c-807-17.jpg,807,17,0
205595,c-807-18.jpg,807,18,0
205596,c-807-19.jpg,807,19,0
205597,c-807-20.jpg,807,20,0


# TRAIN AND TEST ON THE EXPLOITATION SET

In [120]:
test_gens_preds = []
test_fakes_preds = []

train_idx, test_idx = train_test_split(np.arange(1,25), train_size=0.5)
exp_df = data_f.loc[data_f['user_id'].isin(exp_user_ids)]
exp_vf = visual_f.loc[exp_df.index]
exp_df_gen = exp_df.loc[exp_df['fakeness']==0]
exp_df_fake = exp_df.loc[exp_df['fakeness']==1]
exp_df_fake_10 = exp_df_fake.loc[exp_df_fake['sig_id'].isin(choice(np.arange(1,31),10,replace=False))]
exp_df_gen_12 = exp_df_gen.loc[exp_df_gen['sig_id'].isin(train_idx)]
exp_df_valid_gen_10 = exp_df_gen.loc[exp_df_gen['sig_id'].isin(test_idx[0:10])]
exp_df_gen_10 = exp_df_gen.loc[exp_df_gen['sig_id'].isin(train_idx[:-2])]

dev_val_df = data_f.loc[data_f['user_id'].isin(dev_val_user_ids)]
dev_val_vf = visual_f.loc[dev_val_df.index]
dev_val_df_gen = dev_val_df.loc[dev_val_df['fakeness']==0]
dev_val_df_valid_gen_14 = dev_val_df_gen.loc[dev_val_df_gen['sig_id'].isin(choice(np.arange(1,25),14,replace=False))]

c:\users\mert\anaconda3\envs\ml_gpu\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [121]:
exp_df_gen_10

,path,user_id,sig_id,fakeness
17339,c-1201-06.jpg,1201,6,0
17341,c-1201-08.jpg,1201,8,0
17342,c-1201-09.jpg,1201,9,0
17346,c-1201-13.jpg,1201,13,0
17348,c-1201-15.jpg,1201,15,0
17349,c-1201-16.jpg,1201,16,0
17350,c-1201-17.jpg,1201,17,0
17352,c-1201-19.jpg,1201,19,0
17353,c-1201-20.jpg,1201,20,0
17354,c-1201-21.jpg,1201,21,0


In [122]:
for user_id in tqdm.tqdm(exp_user_ids, ascii=True):
    clf = SVC(C=1,gamma='scale',class_weight='balanced', probability=False, kernel=USER_KERNEL)
    y_train = (pd.concat([exp_df_gen_10.loc[exp_df_gen_10['user_id']==user_id],dev_val_df_valid_gen_14.loc[dev_val_df_valid_gen_14['user_id']!=user_id]]))['user_id']==user_id
    X_train = visual_f.loc[y_train.index]  
    clf.fit(X_train, y_train)
    y_valid_fakes = exp_df_fake_10.loc[(exp_df_fake_10['user_id']==user_id)]
    X_valid_f = visual_f.loc[y_valid_fakes.index]
    test_fakes_preds.append(clf.decision_function(X_valid_f))
    y_valid_gens = exp_df_valid_gen_10.loc[exp_df_valid_gen_10['user_id']==user_id]
    X_valid_g = visual_f.loc[y_valid_gens.index]
    test_gens_preds.append(clf.decision_function(X_valid_g))



  0%|                                                                                         | 0/2800 [00:00<?, ?it/s]

  0%|                                                                               | 1/2800 [00:03<2:27:40,  3.17s/it]

  0%|                                                                               | 2/2800 [00:06<2:27:14,  3.16s/it]

  0%|                                                                               | 3/2800 [00:10<2:37:38,  3.38s/it]

  0%|1                                                                              | 4/2800 [00:13<2:40:32,  3.45s/it]

  0%|1                                                                              | 5/2800 [00:16<2:25:32,  3.12s/it]

  0%|1                                                                              | 6/2800 [00:19<2:23:33,  3.08s/it]

  0%|1                                                                              | 7/2800 [00:23<2:34:21,  3.32s/it]

  0%|2                        

  2%|#8                                                                            | 67/2800 [03:35<2:23:28,  3.15s/it]

  2%|#8                                                                            | 68/2800 [03:38<2:28:38,  3.26s/it]

  2%|#9                                                                            | 69/2800 [03:42<2:26:14,  3.21s/it]

  2%|#9                                                                            | 70/2800 [03:45<2:27:27,  3.24s/it]

  3%|#9                                                                            | 71/2800 [03:47<2:11:42,  2.90s/it]

  3%|##                                                                            | 72/2800 [03:50<2:17:57,  3.03s/it]

  3%|##                                                                            | 73/2800 [03:53<2:14:05,  2.95s/it]

  3%|##                                                                            | 74/2800 [03:56<2:15:38,  2.99s/it]

  3%|##                         

  5%|###6                                                                         | 134/2800 [07:10<2:28:19,  3.34s/it]

  5%|###7                                                                         | 135/2800 [07:13<2:31:22,  3.41s/it]

  5%|###7                                                                         | 136/2800 [07:17<2:32:40,  3.44s/it]

  5%|###7                                                                         | 137/2800 [07:19<2:18:29,  3.12s/it]

  5%|###7                                                                         | 138/2800 [07:23<2:33:01,  3.45s/it]

  5%|###8                                                                         | 139/2800 [07:27<2:34:20,  3.48s/it]

  5%|###8                                                                         | 140/2800 [07:30<2:24:05,  3.25s/it]

  5%|###8                                                                         | 141/2800 [07:34<2:33:25,  3.46s/it]

  5%|###9                       

  7%|#####5                                                                       | 201/2800 [10:40<2:06:55,  2.93s/it]

  7%|#####5                                                                       | 202/2800 [10:44<2:11:32,  3.04s/it]

  7%|#####5                                                                       | 203/2800 [10:47<2:12:18,  3.06s/it]

  7%|#####6                                                                       | 204/2800 [10:50<2:16:58,  3.17s/it]

  7%|#####6                                                                       | 205/2800 [10:53<2:07:39,  2.95s/it]

  7%|#####6                                                                       | 206/2800 [10:55<2:06:03,  2.92s/it]

  7%|#####6                                                                       | 207/2800 [10:58<2:05:03,  2.89s/it]

  7%|#####7                                                                       | 208/2800 [11:02<2:13:43,  3.10s/it]

  7%|#####7                     

 10%|#######3                                                                     | 268/2800 [14:19<2:27:36,  3.50s/it]

 10%|#######3                                                                     | 269/2800 [14:21<2:15:12,  3.21s/it]

 10%|#######4                                                                     | 270/2800 [14:25<2:19:10,  3.30s/it]

 10%|#######4                                                                     | 271/2800 [14:28<2:13:41,  3.17s/it]

 10%|#######4                                                                     | 272/2800 [14:31<2:15:36,  3.22s/it]

 10%|#######5                                                                     | 273/2800 [14:34<2:12:36,  3.15s/it]

 10%|#######5                                                                     | 274/2800 [14:38<2:18:02,  3.28s/it]

 10%|#######5                                                                     | 275/2800 [14:41<2:16:12,  3.24s/it]

 10%|#######5                   

...

 89%|###################################################################2        | 2479/2800 [2:22:29<19:59,  3.74s/it]

 89%|###################################################################3        | 2480/2800 [2:22:32<18:42,  3.51s/it]

 89%|###################################################################3        | 2481/2800 [2:22:36<19:51,  3.74s/it]

 89%|###################################################################3        | 2482/2800 [2:22:39<19:15,  3.63s/it]

 89%|###################################################################3        | 2483/2800 [2:22:43<19:05,  3.61s/it]

 89%|###################################################################4        | 2484/2800 [2:22:48<20:34,  3.91s/it]

 89%|###################################################################4        | 2485/2800 [2:22:50<18:32,  3.53s/it]

 89%|###################################################################4        | 2486/2800 [2:22:53<17:46,  3.40s/it]

 89%|###########################

 91%|#####################################################################1      | 2546/2800 [2:26:40<14:50,  3.51s/it]

 91%|#####################################################################1      | 2547/2800 [2:26:44<14:57,  3.55s/it]

 91%|#####################################################################1      | 2548/2800 [2:26:47<14:46,  3.52s/it]

 91%|#####################################################################1      | 2549/2800 [2:26:51<15:14,  3.64s/it]

 91%|#####################################################################2      | 2550/2800 [2:26:55<14:58,  3.59s/it]

 91%|#####################################################################2      | 2551/2800 [2:26:58<15:17,  3.69s/it]

 91%|#####################################################################2      | 2552/2800 [2:27:02<14:58,  3.62s/it]

 91%|#####################################################################2      | 2553/2800 [2:27:05<14:13,  3.45s/it]

 91%|###########################

 93%|######################################################################9     | 2613/2800 [2:30:47<11:14,  3.61s/it]

 93%|######################################################################9     | 2614/2800 [2:30:51<11:08,  3.59s/it]

 93%|######################################################################9     | 2615/2800 [2:30:55<11:11,  3.63s/it]

 93%|#######################################################################     | 2616/2800 [2:30:57<10:00,  3.27s/it]

 93%|#######################################################################     | 2617/2800 [2:31:01<10:34,  3.47s/it]

 94%|#######################################################################     | 2618/2800 [2:31:05<10:38,  3.51s/it]

 94%|#######################################################################     | 2619/2800 [2:31:08<10:14,  3.40s/it]

 94%|#######################################################################1    | 2620/2800 [2:31:13<11:34,  3.86s/it]

 94%|###########################

 96%|########################################################################7   | 2680/2800 [2:34:51<07:09,  3.58s/it]

 96%|########################################################################7   | 2681/2800 [2:34:54<06:44,  3.40s/it]

 96%|########################################################################7   | 2682/2800 [2:34:56<06:18,  3.20s/it]

 96%|########################################################################8   | 2683/2800 [2:35:01<06:46,  3.48s/it]

 96%|########################################################################8   | 2684/2800 [2:35:03<06:16,  3.25s/it]

 96%|########################################################################8   | 2685/2800 [2:35:06<05:44,  3.00s/it]

 96%|########################################################################9   | 2686/2800 [2:35:08<05:26,  2.87s/it]

 96%|########################################################################9   | 2687/2800 [2:35:11<05:16,  2.80s/it]

 96%|###########################

 98%|##########################################################################5 | 2747/2800 [2:38:55<03:01,  3.43s/it]

 98%|##########################################################################5 | 2748/2800 [2:38:59<03:05,  3.56s/it]

 98%|##########################################################################6 | 2749/2800 [2:39:02<03:00,  3.53s/it]

 98%|##########################################################################6 | 2750/2800 [2:39:06<03:06,  3.72s/it]

 98%|##########################################################################6 | 2751/2800 [2:39:10<03:02,  3.73s/it]

 98%|##########################################################################6 | 2752/2800 [2:39:14<03:05,  3.87s/it]

 98%|##########################################################################7 | 2753/2800 [2:39:18<03:00,  3.84s/it]

 98%|##########################################################################7 | 2754/2800 [2:39:21<02:46,  3.62s/it]

 98%|###########################

In [123]:
flat_test_fakes_preds = np.expand_dims(np.array([item for sublist in test_fakes_preds for item in sublist]),axis=1)
flat_test_gens_preds = np.expand_dims(np.array([item for sublist in test_gens_preds for item in sublist]),axis=1)
print("____At the EER threshold decided on the Validation set____")
print("FRR : ",(1-len(flat_test_gens_preds[flat_test_gens_preds>=glob_th])/len(flat_test_gens_preds))*100)
print("FARskilled : ",(1-len(flat_test_fakes_preds[flat_test_fakes_preds<glob_th])/len(flat_test_fakes_preds))*100)

____At the EER threshold decided on the Validation set____
FRR :  7.296428571428571
FARskilled :  6.7071428571428555


In [124]:
all_test_preds = np.vstack((flat_test_fakes_preds,flat_test_gens_preds))
all_test_labels = np.vstack((np.zeros((flat_test_fakes_preds.shape[0],1)),np.ones((flat_test_gens_preds.shape[0],1))))

fpr,tpr,threshold = roc_curve(all_test_labels,all_test_preds)
fnr = 1 - tpr
EER = fpr[np.nanargmin(np.absolute((fnr - fpr)))]
eer_th = threshold[np.nanargmin(np.absolute((fnr - fpr)))]
print('EER_glob for test set: ', EER*100,'\nEER_Threshold_glob for test set: ', eer_th)

EER_glob for test set:  6.950000000000001 
EER_Threshold_glob for test set:  0.03830680345687121


In [125]:
assert len(test_fakes_preds)==len(test_gens_preds)
EER_accum=0
for idx,val in enumerate(test_fakes_preds):
    user_test_fakes_preds = np.expand_dims(np.array(test_fakes_preds[idx]),axis=1)
    user_test_gens_preds = np.expand_dims(np.array(test_gens_preds[idx]),axis=1)
    all_user_test_preds = np.vstack((user_test_fakes_preds,user_test_gens_preds))
    all_user_test_labels = np.vstack((np.zeros((user_test_fakes_preds.shape[0],1)),np.ones((user_test_gens_preds.shape[0],1)))) 
    fpr,tpr,threshold = roc_curve(all_user_test_labels,all_user_test_preds)
    fnr = 1 - tpr
    EER = fpr[np.nanargmin(np.absolute((fnr - fpr)))]
    EER_accum += EER
    
print('EER_user for test set : ', (EER_accum*100)/len(test_fakes_preds))

EER_user for test set :  2.646428571428578
